In [937]:
import pandas as pd
import numpy as np
import scipy.stats as st
filename = r'table.xls'


usage: ipykernel_launcher.py [-h] file
ipykernel_launcher.py: error: unrecognized arguments: -f
C:\Users\User\.conda\envs\SkyTec_test\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


SystemExit: 2

In [934]:
# Чтение файла формата xls
xls_frame = pd.ExcelFile(filename).parse(0)
column_names = list(xls_frame.columns)
# Преобразование столбца date из строки в datetime
xls_frame['date'] = pd.to_datetime(xls_frame['date'], format='%Y-%m-%d')
xls_frame

,date,installs,costs,revenue,payers share
0,2021-05-01,5,150,314,5
1,2021-04-24,15,33,159,10
2,2020-12-24,7,421,512,12
3,2020-12-21,3,127,13,7


In [935]:
# Для каждой колонки применяем необходимые функции
# Сумма применяется, чтобы была видна понедельная статистика
funcs = [pd.NamedAgg(column='sum', aggfunc=np.sum), 
         pd.NamedAgg(column='mean', aggfunc=np.mean),
         pd.NamedAgg(column='std', aggfunc=np.std),
         pd.NamedAgg(column='var', aggfunc=st.variation),
         pd.NamedAgg(column='med', aggfunc=np.median),
         pd.NamedAgg(column='q25', aggfunc=lambda x: np.percentile(x, 25)),
         pd.NamedAgg(column='q75', aggfunc=lambda x: np.percentile(x, 75)),
         ]
x = xls_frame.groupby([pd.Grouper(key='date', freq='w')]).agg(funcs)
x = x.loc[(x.loc[:, pd.IndexSlice[:, 'sum']] != 0).any(axis=1)].reset_index()
x

date installs                                        costs         \
                  sum  mean       std  var   med   q25   q75   sum   mean   
0 2020-12-27       10   5.0  2.828427  0.4   5.0   4.0   6.0   548  274.0   
1 2021-04-25       15  15.0       NaN  0.0  15.0  15.0  15.0    33   33.0   
2 2021-05-02        5   5.0       NaN  0.0   5.0   5.0   5.0   150  150.0   

   ... revenue                 payers share                                  \
   ...     med     q25     q75          sum  mean       std       var   med   
0  ...   262.5  137.75  387.25           19   9.5  3.535534  0.263158   9.5   
1  ...   159.0  159.00  159.00           10  10.0       NaN  0.000000  10.0   
2  ...   314.0  314.00  314.00            5   5.0       NaN  0.000000   5.0   

                 
     q25    q75  
0   8.25  10.75  
1  10.00  10.00  
2   5.00   5.00  

[3 rows x 29 columns]

In [936]:
# нахождение корреляции по группам, сформированным по недельным промежуткам
corr = lambda a: np.corrcoef(a['installs'], a['revenue']-a['costs'])[0, 1]
rdf = pd.DataFrame(xls_frame)
# сперва находим корреляцию к группах и записываем в новую колонку r, затем группируем по дате, оставляя последнее значение группы 
rdf = rdf.groupby([pd.Grouper(key='date', freq='w')]).apply(lambda b: b.assign(r=corr(b))).groupby([pd.Grouper(key='date', freq='w')]).last()
rdf = rdf.loc[(~np.isnan(rdf)).any(axis=1), :].reset_index()
x['r'] = rdf['r']
x

C:\Users\User\.conda\envs\SkyTec_test\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\User\.conda\envs\SkyTec_test\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\Users\User\.conda\envs\SkyTec_test\lib\site-packages\numpy\lib\function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\User\.conda\envs\SkyTec_test\lib\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\User\.conda\envs\SkyTec_test\lib\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


date installs                                        costs         \
                  sum  mean       std  var   med   q25   q75   sum   mean   
0 2020-12-27       10   5.0  2.828427  0.4   5.0   4.0   6.0   548  274.0   
1 2021-04-25       15  15.0       NaN  0.0  15.0  15.0  15.0    33   33.0   
2 2021-05-02        5   5.0       NaN  0.0   5.0   5.0   5.0   150  150.0   

   ... revenue         payers share                                         \
   ...     q25     q75          sum  mean       std       var   med    q25   
0  ...  137.75  387.25           19   9.5  3.535534  0.263158   9.5   8.25   
1  ...  159.00  159.00           10  10.0       NaN  0.000000  10.0  10.00   
2  ...  314.00  314.00            5   5.0       NaN  0.000000   5.0   5.00   

            r  
     q75       
0  10.75  1.0  
1  10.00  NaN  
2   5.00  NaN  

[3 rows x 30 columns]